In [12]:
import os
import glob
import plotly
import osmnx as ox
import plotly.express as px
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ipypb import track
from shapely.geometry import Polygon, MultiPolygon

In [11]:
Data_Path = "C:\\Users\\sahit\\ADES\\ADES\\DataSource"

In [14]:
bridges = pd.read_excel(Data_Path + '\\BMMS_overview_SortedRoad.xlsx')